In [1]:
import astropy.units as u
import astropy.constants as c

from ipynb.fs.full.Wind_accretion import *
from ipynb.fs.full.Mass_accumulation import L_tot, find_t_GiantStage

In [2]:
def hasHotComp_mask_G_WD_unstable(Data, mask_G_WD):

    SWs = Data['BSE_Switch_Log']  # Access the switch log data
        
    # Very sketchy but I make the last one False because I have no way to calculate its lifetime
    mask_G_WD[-1] = False
    
    mask_plus_one = [False] * len(mask_G_WD)
    for i in range(len(mask_G_WD)):
        if mask_G_WD[i]:
            
            mask_plus_one[i + 1] = True
    
    t_GiantStage  = np.array(SWs['Time'][()][mask_plus_one] - SWs['Time'][()][mask_G_WD]) * u.Myr
        
    # A better way to do the next thing is (probably) to control that the Seed of the next switch-state is the same
    t_GiantStage[t_GiantStage < 0] = 0
    
    # Stellar properties
    Type_G          = SWs['Stellar_Type(1)'][()][mask_G_WD]
    Type_WD         = SWs['Stellar_Type(2)'][()][mask_G_WD]
    Mass_G          = SWs['Mass(1)'][()][mask_G_WD]         * u.Msun
    Mass_WD         = SWs['Mass(2)'][()][mask_G_WD]         * u.Msun
    Radius_G        = SWs['Radius(1)'][()][mask_G_WD]       * u.Rsun
    Radius_WD       = SWs['Radius(2)'][()][mask_G_WD]       * u.Rsun
    Luminosity_G    = SWs['Luminosity(1)'][()][mask_G_WD]   * u.Lsun
    Luminosity_WD   = SWs['Luminosity(2)'][()][mask_G_WD]   * u.Lsun
        
    # Binary properties
    a      = SWs['SemiMajorAxis'][()][mask_G_WD] * u.Rsun
    
    v_orb = np.sqrt( c.G * (Mass_G + Mass_WD) / a ).to(u.km / u.s)

        
     ########### Defining parameters ###########

    # timesteps which we will evaluate
    t_Gs = np.linspace(0, t_GiantStage, 500)
    dt   = t_GiantStage/(501)
                        
    # time that the system is visible as a Symbiotic system
    t_visible = np.zeros(len(Mass_WD)) * u.Myr
    
    # H built up on the surface of the WD    
    Delta_M = np.zeros(len(Mass_WD)) * u.Msun
    
    # Time that the WD is on the plateau and time is takes until 
    t_on = np.zeros(len(Mass_WD)) * u.Myr
    t_cool = np.zeros(len(Mass_WD)) * u.Myr

    # Luminosity due to H-burning
    L_on = np.zeros(len(Mass_WD)) * u.Lsun
    
    # Built up mass necessary for SyNe
    M_crit = ign_mass(Mass_WD).to(u.Msun)
    
    # Massloss of the giants
    Mdot = interpWindLossALL_t(Type_G, Mass_G, t_Gs)
        
    #____________________________________________________________________        

    for t_ind in range(len(t_Gs)):
                
        t = t_Gs[t_ind]
                
        Mdotacc = Mdot_acc(Mass_WD, a, v_orb, Mdot[:,t_ind], Radius_G, Mass_G).to(u.Msun / u.yr)
        
        Mdotaccum = Mdot_accum(Mdotacc, Mass_WD).to(u.Msun / u.yr)

        Delta_M += (Mdotaccum * dt).to(u.Msun)
        Delta_M -= (radStellarWind(L_on + Luminosity_WD, Mass_WD, Radius_WD) * dt).to(u.Msun)

        Delta_M[Delta_M < 0] = 0
        #____________________________________________________________________
                
        # Stable H-burning
        stableSyS = isStable(Mass_WD, Mdotacc)
            
        # Beginning a SyN
        startSyNe    = (Delta_M > M_crit)
        
        # start burning / still burning from earlier
        startBurning = (stableSyS | startSyNe)
        stillBurning = (t < t_on)
        
        # Time until it is not on the plateau anymore and time until it is not 
        t_on[startBurning]   = t[startBurning] + t_on_fun(Mdotacc[startBurning], Mass_WD[startBurning], Radius_WD[startBurning]) # t in Myr
        
        # t_cool NEEDS TO BE LOOKED AT AGAIN
        t_cool[startBurning] = t_on[startBurning] + t_cool_fun(Mass_WD[startBurning], Luminosity_WD[startBurning], t_GiantStage[startBurning]) # t in Myr
        
        isBurning    = startBurning | stillBurning
        isSyNe       = isBurning & ~stableSyS
        
        #____________________________________________________________________

        Delta_M[isBurning] = 0
    
    
        L_on[startBurning] = L_H(Mass_WD[startBurning])
        L_on[t > t_on] = L_tot(Luminosity_WD[t > t_on], L_on[t > t_on], t[t > t_on], t_on[t > t_on])
        
        #____________________________________________________________________
    
        visible = (L_on + Luminosity_WD + L_grav(Mass_WD, Radius_WD, Mdotacc) > 10 * u.Lsun)
                
        t_visible[isSyNe & visible] += np.min([dt[isSyNe & visible],t_cool[isSyNe & visible] - t[isSyNe & visible]],0) * u.Myr
            
    return t_visible   

In [3]:
def hasHotComp_mask_WD_G_unstable(Data, mask_WD_G):

    SWs = Data['BSE_Switch_Log']  # Access the switch log data
        
    # Very sketchy but I make the last one False because I have no way to calculate its lifetime
    mask_WD_G[-1] = False
    
    mask_plus_one = [False] * len(mask_WD_G)
    for i in range(len(mask_WD_G)):
        if mask_WD_G[i]:
            
            mask_plus_one[i + 1] = True
    
    t_GiantStage  = np.array(SWs['Time'][()][mask_plus_one] - SWs['Time'][()][mask_WD_G]) * u.Myr
        
    # A better way to do the next thing is (probably) to control that the Seed of the next switch-state is the same
    t_GiantStage[t_GiantStage < 0] = 0
    
    # Stellar properties
    Type_WD        = SWs['Stellar_Type(1)'][()][mask_WD_G]
    Type_G         = SWs['Stellar_Type(2)'][()][mask_WD_G]
    Mass_WD        = SWs['Mass(1)'][()][mask_WD_G]         * u.Msun
    Mass_G         = SWs['Mass(2)'][()][mask_WD_G]         * u.Msun
    Radius_WD      = SWs['Radius(1)'][()][mask_WD_G]       * u.Rsun
    Radius_G       = SWs['Radius(2)'][()][mask_WD_G]       * u.Rsun
    Luminosity_WD  = SWs['Luminosity(1)'][()][mask_WD_G]   * u.Lsun
    Luminosity_G   = SWs['Luminosity(2)'][()][mask_WD_G]   * u.Lsun
        
    # Binary properties
    a      = SWs['SemiMajorAxis'][()][mask_WD_G] * u.Rsun
    
    v_orb = np.sqrt( c.G * (Mass_G + Mass_WD) / a ).to(u.km / u.s)

        
     ########### Defining parameters ###########

    # timesteps which we will evaluate
    t_Gs = np.linspace(0, t_GiantStage, 500)
    dt   = t_GiantStage/(501)
                        
    # time that the system is visible as a Symbiotic system
    t_visible = np.zeros(len(Mass_WD)) * u.Myr
    
    # H built up on the surface of the WD    
    Delta_M = np.zeros(len(Mass_WD)) * u.Msun
    
    # Time that the WD is on the plateau and time is takes until 
    t_on = np.zeros(len(Mass_WD)) * u.Myr
    t_cool = np.zeros(len(Mass_WD)) * u.Myr

    # Luminosity due to H-burning
    L_on = np.zeros(len(Mass_WD)) * u.Lsun
    
    # Built up mass necessary for SyNe
    M_crit = ign_mass(Mass_WD).to(u.Msun)
    
    # Massloss of the giants
    Mdot = interpWindLossALL_t(Type_G, Mass_G, t_Gs)
        
    #____________________________________________________________________        

    for t_ind in range(len(t_Gs)):
                
        t = t_Gs[t_ind]
                
        Mdotacc = Mdot_acc(Mass_WD, a, v_orb, Mdot[:,t_ind], Radius_G, Mass_G).to(u.Msun / u.yr)
        
        Mdotaccum = Mdot_accum(Mdotacc, Mass_WD).to(u.Msun / u.yr)

        Delta_M += (Mdotaccum * dt).to(u.Msun)
        Delta_M -= (radStellarWind(L_on + Luminosity_WD, Mass_WD, Radius_WD) * dt).to(u.Msun)

        Delta_M[Delta_M < 0] = 0
        #____________________________________________________________________
                
        # Stable H-burning
        stableSyS = isStable(Mass_WD, Mdotacc)
            
        # Beginning a SyN
        startSyNe    = (Delta_M > M_crit)
                
        # start burning / still burning from earlier
        startBurning = (stableSyS | startSyNe)
        stillBurning = (t < t_on)
        
        # Time until it is not on the plateau anymore and time until it is not 
        t_on[startBurning]   = t[startBurning] + t_on_fun(Mdotacc[startBurning], Mass_WD[startBurning], Radius_WD[startBurning]) # t in Myr
        
        # t_cool NEEDS TO BE LOOKED AT AGAIN
        t_cool[startBurning] = t_on[startBurning] + t_cool_fun(Mass_WD[startBurning], Luminosity_WD[startBurning], t_GiantStage[startBurning]) # t in Myr

        isBurning    = startBurning | stillBurning
        isSyNe       = isBurning & ~stableSyS
        
        #____________________________________________________________________

        Delta_M[isBurning] = 0
    
    
        L_on[startBurning] = L_H(Mass_WD[startBurning])
        L_on[t > t_on] = L_tot(Luminosity_WD[t > t_on], L_on[t > t_on], t[t > t_on], t_on[t > t_on])
        
        #____________________________________________________________________
    
        visible = (L_on + Luminosity_WD + L_grav(Mass_WD, Radius_WD, Mdotacc) > 10 * u.Lsun)
                
        t_visible[isSyNe & visible] += np.min([dt[isSyNe & visible],t_cool[isSyNe & visible] - t[isSyNe & visible]],0) * u.Myr
            
    return t_visible   